# Python Machine Learning - decision_tree

In [1]:
%load_ext watermark
%watermark -a "Sebastian Raschka" -u -d -p numpy,pandas,matplotlib,sklearn

Sebastian Raschka 
last updated: 2020-11-06 

numpy 1.18.5
pandas 1.0.5
matplotlib 3.2.2
sklearn 0.23.1


In [1]:
from IPython.display import Image
%matplotlib inline

###  加载数据集

使用银行营销数据集，这些数据共包括16个特征，输出标签有2个（“是”、“否”），即客户是否已订阅定期存款。我们将采用其中的三个特征“年龄”、“年平均余额”以及“该月与该客户最后一次联系的日期”对客户是否订阅定期存款来进行预测。

**属性信息:**

<p> 银行客户数据：</p>
    
- age 年龄（数字）
- job 职务：职务类型（类别：“管理员”，“蓝领”，“企业家”，“女佣”，“管理”，“退休” ，“自雇”，“服务”，“学生”，“技术员”，“待业”，“未知”）
- marital 婚姻：婚姻状况（类别：“已婚”，“离婚”，“单身”，“未知”）
- education 教育（类别："未知", "中学", "小学", "高等教育"）
- default 违约：信用违约吗？（类别：“否”，”是'）
- balance 收支：年平均余额
- housing 住房：有住房贷款吗？（分类：“否”，“是”）
- loan 贷款：有个人贷款吗？（类别：“否”，“是”）

<p>与当前广告系列的最后一次接触相关的：</p>

- contact 联系人：联系人通讯类型（类别：“未知”，“蜂窝网络”，“电话”）
- day 日期：这个月最后一次联系的日期
- month 月分：一年中的最后联系人月份（类别：“ jan”，“ feb”，“ mar'，...，'nov'，'dec'）
- duration 持续时间：最后一次接触持续时间，以秒为单位（数字）重要说明：此属性会极大影响输出目标（例如，如果duration = 0，则y ='no'）。然而，在执行呼叫之前，持续时间是未知的。同样，在通话结束后，y显然是已知的。从而，此输入仅应出于基准目的而包括在内，如果要使用一个现实的预测模型，则应将其丢弃。
    
<p> 其他属性：</p>

- campaign 广告活动：在该广告活动期间和该客户执行的联系数量（数字，包括最后联系人）
- pdays：从上一次广告活动最后一次联系客户以来经过的天数（数字； 999表示客户不是先接触）
- precious 以前的：这次活动之前，与该客户进行接触次数
- poutcome 上一次市场活动的结果：以前的营销活动（分类的结果：“失败”，“成功”，“未知”，“其他”）


**输出类别信息:**
- 客户是否已订阅定期存款？（二进制：“是”，“否”）

In [1]:
from sklearn import datasets
import numpy as np
import pandas as pd

bm = pd.read_csv('./bank/bank.txt',sep=';')


X = bm[['age','balance','day']].values[:200,:]
y = bm['y'].values[:200]

bm.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


将数据集划为70%作为训练集，30%划分为测试集。

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y)

### 使用scikit-learn训练一个决策树

In [3]:
from sklearn.tree import DecisionTreeClassifier

# 采用gini系数进行度量，树的最大深度设置为4
tree = DecisionTreeClassifier(criterion='gini', 
                              max_depth=4, 
                              random_state=1)
# 训练
tree.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=4, random_state=1)

# DecisionTreeClassifier()函数参数
class sklearn.tree.DecisionTreeClassifier(criterion=’gini’, splitter=’best’, max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, class_weight=None, presort=False)

criterion：默认gini，即CART算法。

max_depth：决策树最大深度，默认值是‘None’。一般数据比较少或者特征少的时候可以不用管这个值，如果模型样本数量多，特征也多时，推荐限制这个最大深度，具体取值取决于数据的分布。常用的可以取值10-100之间，常用来解决过拟合。

random_state：是用来设置决策树分枝中随机模式的参数，在高维度时sklearn决策树的特征随机性会很明显，低维度的数据（比如鸢尾花数据集），随机性几乎不会显现。高维数据下我们设置random_state并配合splitter参数可以让模型稳定下来，保证同一数据集下是决策树结果可以多次复现，便于模型参数优化。


对test中的样本进行测试：

In [4]:
test_sample = X_test[:3,:]
print(test_sample)
pre_sample = tree.predict(test_sample)
print(pre_sample)

[[ 39 501  26]
 [ 28  80  20]
 [ 37   0  26]]
['no' 'no' 'no']


### 模型准确率：

In [5]:
from sklearn.metrics import accuracy_score
y_train_pre = tree.predict(X_train)
y_test_pre = tree.predict(X_test)
print("training accuracy: %.3f"%accuracy_score(y_train_pre, y_train))
print("testing accuracy: %.3f"%accuracy_score(y_test_pre, y_test))

training accuracy: 0.907
testing accuracy: 0.817


**使用Graphviz库进行对决策树进行可视化：**
<p>scikit-learn允许将训练后得到的决策树导出为.dot格式的文件，这使得我们可以使用GraphViz程序进行可视化处理。</p>

In [13]:
from pydotplus import graph_from_dot_data
from sklearn.tree import export_graphviz

# 生成.dot文件
dot_data = export_graphviz(tree,
                           filled=True, 
                           rounded=True,
                           class_names=['yes', 
                                        'no',],
                           feature_names=['年龄', 
                                          '收支',
                                          '联系日期'],
                           out_file=None) 
# export_graphviz(tree, out_file='tree.dot',feature_names=['age','balance','duration'])
# 将.dot文件存为图像文件
graph = graph_from_dot_data(dot_data) 
# 将图像文件写入
graph.write_png('tree.png')


True

In [15]:
# Image(filename='tree.png')

NameError: name 'Image' is not defined

In [17]:
from Ipython import display
display.Image(filename='tree.png',width=600)


ModuleNotFoundError: No module named 'Ipython'

In [18]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

img = cv2.imread("D:\2022秋冬\前沿数学专题讨论\tree.png")#读取的是bgr格式
# img = cv2.imdecode(np.fromfile(img_path,dtype=np.uint8),-1) #读取带中文图片
# print(img.shape)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(img)
plt.show()


error: OpenCV(4.6.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
